In [1]:
%pip install lightning

  Using cached lightning-2.5.1-py3-none-any.whl.metadata (39 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached pytorch_lightning-2.5.1-py3-none-any.whl.metadata (20 kB)
  Using cached aiohttp-3.11.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.6.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached multidict-6.4.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached yarl-1.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (72 kB)
U

In [2]:
%pip install -U 'wandb>=0.12.10'

  Using cached wandb-0.19.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached sentry_sdk-2.26.1-py2.py3-none-any.whl.metadata (10 kB)
  Using cached setproctitle-1.3.5-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached wandb-0.19.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.9 MB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Using cached sentry_sdk-2.26.1-py2.py3-none-any.whl (340 kB)
Using cached setproctitle-1.3.5-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torchvision.models
import torchvision.transforms as transforms

from datapipeline.asl_image_data_module import ASLImageDataModule
from lightning.pytorch.loggers import WandbLogger
from models.asl_model import ASLModel
from models.training import train, PROJECT_NAME, ENTITY_NAME

In [4]:
img_size = 400

data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
datamodule = ASLImageDataModule(path="/exchange/dspro2/silent-speech/ASL_Pictures_Dataset", transforms=data_transforms)

In [5]:
class ASLCNN_img(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3), # 400 -> 200
            nn.ReLU(),
            nn.MaxPool2d(2),  # 200 -> 100
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 100
            nn.ReLU(),
            nn.MaxPool2d(2),  # 100 -> 50
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 50
            nn.ReLU(),
            nn.MaxPool2d(2),  # 50 -> 25
        
            nn.Flatten(),
            nn.Linear(128 * 25 * 25, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 28)  # number of classes
        )

    def forward(self, x):
        x = self.model(x)
        return x



In [6]:
torch_model = ASLCNN_img()
model = ASLModel(model=torch_model, criterion=nn.CrossEntropyLoss(), optimizer=torch.optim.Adam(torch_model.parameters(), lr=1e-4))

logger = WandbLogger(name="ASL_CNN-1", project=PROJECT_NAME)

train(model, datamodule, logger=logger)

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shse13 (shse13-doe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ASLCNN_img         | 41.1 M | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | valid_accuracy | MulticlassAccuracy | 0      | train
4 | test_accuracy  | MulticlassAccuracy | 0      | train
--------------------------------------------------------------
41.1 M    Trainable params
0         Non-trainable params
41.1 M    Total params
164.288   Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric train_accuracy improved by 0.057 >= min_delta = 0.0. New best score: 0.854


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric valid_accuracy did not improve in the last 5 records. Best score: 0.036. Signaling Trainer to stop.
wandb: Adding directory to artifact (./silent-speech/sbbk3tcc/checkpoints)... Done. 1.2s


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▄▄▄▄▄▄▄▄▅▅▇▇▇▇▇▇▇▇▇▇█████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-Adam-momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-Adam-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy_epoch,▁▅▆▇██
train_accuracy_step,█▁█▁▁▁▁▁█▂██▂██████████████▁█▇▄▁██▁███▆▁
train_loss_epoch,█▄▂▂▁▁
train_loss_step,▃▄▁▁▁▁▁▁▃▁▆▃▁▂▄▃▁█▂▁▁▁▂▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
trainer/global_step,▁▁▁▁▆▂▂▂▂▂▃██▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇
valid_accuracy_epoch,▁▁▁▁▁▁
valid_accuracy_step,▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█


In [1]:
def training_procedure():
    import wandb
    import torch.nn as nn
    import torch
    from lightning.pytorch.loggers import WandbLogger
    from train import train, PROJECT_NAME
    from models.asl_model import ASLModel
    from models.asl_cnn_img import ASLCNN_img  # adjust import if needed
    from my_data_module import MyDataModule  # adjust to your actual datamodule

    # Access current sweep config
    config = wandb.config

    # Model
    torch_model = ASLCNN_img()
    optimizer = torch.optim.Adam(torch_model.parameters(), lr=config.lr)
    model = ASLModel(model=torch_model, criterion=nn.CrossEntropyLoss(), optimizer=optimizer)

    # Data
    datamodule = MyDataModule(batch_size=config.batch_size)

    # Logger
    logger = WandbLogger(name=f"CNN-sweep-{wandb.run.id}", project=PROJECT_NAME)

    # Train
    train(model, datamodule, logger=logger)


In [2]:
sweep_config = {
    "method": "random",
    "metric": {"name": "valid_accuracy", "goal": "maximize"},
    "parameters": {
        "lr": {
            "min": 1e-5,
            "max": 1e-3
        },
    }
}

sweep(sweep_config, count=20, training_procedure=training_procedure)

NameError: name 'sweep' is not defined